In [176]:
import sys
sys.path.append("..")

In [258]:
from collections import Counter, defaultdict

from time import time

from resources.utils import get_puzzle_input

### Part 1

The Elves play this game by taking turns arranging the marbles in a circle according to very particular rules. The marbles are numbered starting with 0 and increasing by 1 until every marble has a number.

First, the marble numbered 0 is placed in the circle. At this point, while it contains only a single marble, it is still a circle: the marble is both clockwise from itself and counter-clockwise from itself. This marble is designated the current marble.

Then, each Elf takes a turn placing the lowest-numbered remaining marble into the circle between the marbles that are 1 and 2 marbles clockwise of the current marble. (When the circle is large enough, this means that there is one marble between the marble that was just placed and the current marble.) The marble that was just placed then becomes the current marble.

However, if the marble that is about to be placed has a number which is a multiple of 23, something entirely different happens. First, the current player keeps the marble they would have placed, adding it to their score. In addition, the marble 7 marbles counter-clockwise from the current marble is removed from the circle and also added to the current player's score. The marble located immediately clockwise of the marble that was removed becomes the new current marble.

For example, suppose there are 9 players. After the marble with value 0 is placed in the middle, each player (shown in square brackets) takes a turn. The result of each of those turns would produce circles of marbles like this, where clockwise is to the right and the resulting current marble is in parentheses:

```
[-] (0)
[1]  0 (1)
[2]  0 (2) 1 
[3]  0  2  1 (3)
[4]  0 (4) 2  1  3 
[5]  0  4  2 (5) 1  3 
[6]  0  4  2  5  1 (6) 3 
[7]  0  4  2  5  1  6  3 (7)
[8]  0 (8) 4  2  5  1  6  3  7 
[9]  0  8  4 (9) 2  5  1  6  3  7 
[1]  0  8  4  9  2(10) 5  1  6  3  7 
[2]  0  8  4  9  2 10  5(11) 1  6  3  7 
[3]  0  8  4  9  2 10  5 11  1(12) 6  3  7 
[4]  0  8  4  9  2 10  5 11  1 12  6(13) 3  7 
[5]  0  8  4  9  2 10  5 11  1 12  6 13  3(14) 7 
[6]  0  8  4  9  2 10  5 11  1 12  6 13  3 14  7(15)
[7]  0(16) 8  4  9  2 10  5 11  1 12  6 13  3 14  7 15 
[8]  0 16  8(17) 4  9  2 10  5 11  1 12  6 13  3 14  7 15 
[9]  0 16  8 17  4(18) 9  2 10  5 11  1 12  6 13  3 14  7 15 
[1]  0 16  8 17  4 18  9(19) 2 10  5 11  1 12  6 13  3 14  7 15 
[2]  0 16  8 17  4 18  9 19  2(20)10  5 11  1 12  6 13  3 14  7 15 
[3]  0 16  8 17  4 18  9 19  2 20 10(21) 5 11  1 12  6 13  3 14  7 15 
[4]  0 16  8 17  4 18  9 19  2 20 10 21  5(22)11  1 12  6 13  3 14  7 15 
[5]  0 16  8 17  4 18(19) 2 20 10 21  5 22 11  1 12  6 13  3 14  7 15 
[6]  0 16  8 17  4 18 19  2(24)20 10 21  5 22 11  1 12  6 13  3 14  7 15 
[7]  0 16  8 17  4 18 19  2 24 20(25)10 21  5 22 11  1 12  6 13  3 14  7 15
```

The goal is to be the player with the highest score after the last marble is used up. Assuming the example above ends after the marble numbered 25, the winning score is 23+9=32 (because player 5 kept marble 23 and removed marble 9, while no other player got any points in this very short example game).

Here are a few more examples:

```
10 players; last marble is worth 1618 points: high score is 8317
13 players; last marble is worth 7999 points: high score is 146373
17 players; last marble is worth 1104 points: high score is 2764
21 players; last marble is worth 6111 points: high score is 54718
30 players; last marble is worth 5807 points: high score is 37305
```

What is the winning Elf's score?

In [251]:
class CircularBuffer:
    def __init__(self, values, position=0):
        self.buffer = values
        self.position = position
        
    def __len__(self):
        return len(self.buffer)
    
    def move_clockwise(self, distance):
        self.position = self._clockwise(distance)
        return self.position
            
        
    def move_anticlockwise(self, distance):
        self.position = self._anticlockwise(distance)
        return self.position
    
    def add(self, value):
        self.buffer.insert(self.position, value)
        
    def _clockwise(self, distance):
        position = self.position
        remaining_distance = distance
        
        remaining_to_right = len(self.buffer) - position - 1
        if remaining_distance <= remaining_to_right:
            position += remaining_distance
        else:
            position = 0
            remaining_distance -= (remaining_to_right + 1)
            #print(remaining_distance, remaining_distance % len(self.buffer))
            position += remaining_distance % len(self.buffer)
        
        return position

    def _anticlockwise(self, distance):
        position = self.position
        remaining_distance = distance
        
        remaining_to_left = position
        if remaining_distance <= remaining_to_left:
            position -= remaining_distance
        else:
            position = len(self.buffer)
            remaining_distance -= remaining_to_left
            position -= remaining_distance % len(self.buffer)
            
            if position == len(self.buffer):
                position = 0
        
        return position

    def value_anticlockwise(self, distance):
        position = self._anticlockwise(distance)
        return self.buffer[position]

    def remove_anticlockwise(self, distance):
        remove_position = self._anticlockwise(distance)
        if remove_position <= self.position:
            self.position -= 1
        value = self.buffer[remove_position]
        del self.buffer[remove_position]
        return value
    
    def __str__(self):
        vals = []
        for pos in range(len(self.buffer)):
            val = self.buffer[pos]
            if pos == self.position:
                vals.append('({})'.format(val))
            else:
                vals.append(str(val))
            
        return ' '.join(vals)     

In [243]:
circle = CircularBuffer([0, 1, 2, 3], 1)
assert CircularBuffer([0, 1, 2, 3], 1).move_clockwise(7) == 0
assert CircularBuffer([0, 1, 2, 3], 2).move_anticlockwise(7) == 3

In [162]:
CircularBuffer([0, 1, 2, 3], 1)._clockwise(7)


0

In [165]:
circle =CircularBuffer([0])

for x in range(1, 10):
    print(circle.move_clockwise(2))
    circle.add(x)
    print(circle)

0
(1) 0
0
(2) 1 0
2
2 1 (3) 0
0
(4) 2 1 3 0
2
4 2 (5) 1 3 0
4
4 2 5 1 (6) 3 0
6
4 2 5 1 6 3 (7) 0
0
(8) 4 2 5 1 6 3 7 0
2
8 4 (9) 2 5 1 6 3 7 0


In [168]:
circle_pre_remove =CircularBuffer(
    [0, 16, 8, 17, 4, 18, 9, 19, 2, 20, 10, 21, 5, 22, 11, 1, 12, 6, 13, 3, 14, 7, 15],
    13
)
print(circle)

0 16 8 17 4 18 9 19 2 20 10 21 5 (22) 11 1 12 6 13 3 14 7 15


In [174]:
circle_pre_remove =CircularBuffer(
    [0, 16, 8, 17, 4, 18, 9, 19, 2, 20, 10, 21, 5, 22, 11, 1, 12, 6, 13, 3, 14, 7, 15],
    13
)
print(circle_pre_remove)
circle_pre_remove.remove_anticlockwise(7)
circle_pre_remove.move_anticlockwise(6)
print(circle_pre_remove)
circle_pre_remove.move_clockwise(2)
circle_pre_remove.add(24)
print(circle_pre_remove)

0 16 8 17 4 18 9 19 2 20 10 21 5 (22) 11 1 12 6 13 3 14 7 15
0 16 8 17 4 18 (19) 2 20 10 21 5 22 11 1 12 6 13 3 14 7 15
0 16 8 17 4 18 19 2 (24) 20 10 21 5 22 11 1 12 6 13 3 14 7 15


In [ ]:
#0 16 8 17 4 18 (19) 2 20 10 21 5 22 11 1 12 6 13 3 14 7 15
#0 16 8 17 4 18 19 2 (24) 20 10 21 5 22 11 1 12 6 13 3 14 7 15
[5]  0 16  8 17  4 18(19) 2 20 10 21  5 22 11  1 12  6 13  3 14  7 15 
[6]  0 16  8 17  4 18 19  2(24)20 10 21  5 22 11  1 12  6 13  3 14  7 15

In [254]:
def can_enter_fast_mode(remaining_moves, circle):
    max_distance = remaining_moves * 2
    max_distance -= remaining_moves // 23 * 7
    return max_distance < len(circle)

In [250]:
test_circle = CircularBuffer(
    [0, 16, 8, 17, 4, 18, 9, 19, 2, 20, 10, 21, 5, 22, 11, 1, 12, 6, 13, 3, 14, 7, 15],
    13
)
test_circle.remove_anticlockwise(7)
test_circle.move_anticlockwise(6)

can_enter_fast_mode(13, test_circle)

26 22


False

In [262]:
def solve(num_players, num_pieces):
    circle = CircularBuffer([0])
    scores = Counter()
    current_piece = 1
    current_player = 0
    fast_mode = False
    start_time = int(time())
    
    while current_piece <= num_pieces:
        if current_piece % 100000 == 0:
            print('Playing piece {} after {}s...'.format(current_piece, int(time() - start_time)))

        if current_piece % 23 == 0:
            #print(circle)
            if fast_mode:
                # No more deletions
                value = circle.value_anticlockwise(7)
                #scores[current_player] += circle.value_anticlockwise(7)                
            else:
                value = circle.remove_anticlockwise(7)
                #scores[current_player] += circle.remove_anticlockwise(7)
             
            print(current_piece - value)
            scores[current_player] += value

            scores[current_player] += current_piece
            circle.move_anticlockwise(6)
            
            # Can we go into fast mode ignoring deletions and additions
            if not fast_mode:
                entering_fast_mode = can_enter_fast_mode(num_pieces - current_piece, circle)
                if entering_fast_mode:
                    print('Entering fast mode on move {}...'.format(current_piece))
                    fast_mode = True
            
        else:
            circle.move_clockwise(2)
            circle.add(current_piece)
        
        current_piece += 1
        current_player += 1
        if current_player >= num_players:
            current_player = 0
            
    print(len(circle))
    
    return scores.most_common()[0]   

In [259]:
assert solve(9, 25)[1] == 32

#10 players; last marble is worth 1618 points: high score is 8317
#13 players; last marble is worth 7999 points: high score is 146373
#17 players; last marble is worth 1104 points: high score is 2764
#21 players; last marble is worth 6111 points: high score is 54718
#30 players; last marble is worth 5807 points: high score is 37305

assert solve(10, 1618)[1] == 8317
assert solve(13, 7999)[1] == 146373
assert solve(17, 1104)[1] == 2764
assert solve(21, 6111)[1] == 54718
assert solve(30, 5807)[1] == 37305

Entering fast mode on move 23...
24
Entering fast mode on move 1058...
1503
Entering fast mode on move 5221...
7426
Entering fast mode on move 736...
1025
Entering fast mode on move 3979...
5674
Entering fast mode on move 3795...
5391


In [263]:
solve(10, 1618)

14
29
58
77
65
80
95
151
170
131
146
231
244
182
197
212
318
337
248
263
395
411
303
314
329
346
504
365
380
562
703
597
431
446
462
764
482
497
512
846
867
548
563
578
1032
1044
Entering fast mode on move 1058...
614
629
916
1061
665
680
695
1142
1164
731
746
1087
1107
786
797
812
829
1352
848
863
1417
1274
1293
914
1503


(9, 8317)

In [260]:
%%time

# 448 players; last marble is worth 71628 points
print(solve(448, 71628))

Entering fast mode on move 46575...
66490
(170, 394486)
CPU times: user 421 ms, sys: 6.32 ms, total: 427 ms
Wall time: 426 ms


In [261]:
print(solve(448, 71628 * 100))

Playing piece 100000 after 1s...
Playing piece 200000 after 4s...
Playing piece 300000 after 9s...
Playing piece 400000 after 11s...
Playing piece 500000 after 24s...
Playing piece 600000 after 37s...
Playing piece 700000 after 47s...
Playing piece 800000 after 53s...
Playing piece 900000 after 56s...
Playing piece 1000000 after 84s...
Playing piece 1100000 after 131s...
Playing piece 1200000 after 169s...
Playing piece 1300000 after 201s...
Playing piece 1400000 after 229s...
Playing piece 1500000 after 252s...
Playing piece 1600000 after 272s...
Playing piece 1700000 after 289s...
Playing piece 1800000 after 302s...
Playing piece 1900000 after 313s...
Playing piece 2000000 after 320s...
Playing piece 2100000 after 324s...
Playing piece 2200000 after 331s...
Playing piece 2300000 after 473s...


KeyboardInterrupt: 

In [ ]:
max_num = 71628
num_players = 448
for x in range(100):
    print(solve(num_players, max_num))
    max_num += 23

In [217]:
# Given we're adding 23 numbers each time and then moving back 7
# When's the last time we loop around?

circle = CircularBuffer([1])
for x in range(1, 200):
    if x % 23 == 0:
        circle.remove_anticlockwise(7)
        circle.move_anticlockwise(6)
    else:
        circle.move_clockwise(2)
        circle.add(x)
    
    print(circle.position, len(circle) - circle.position)
    



0 2
0 3
2 2
0 5
2 4
4 3
6 2
0 9
2 8
4 7
6 6
8 5
10 4
12 3
14 2
0 17
2 16
4 15
6 14
8 13
10 12
12 11
5 17
7 16
9 15
11 14
13 13
15 12
17 11
19 10
21 9
23 8
25 7
27 6
29 5
31 4
33 3
35 2
0 38
2 37
4 36
6 35
8 34
10 33
12 32
5 38
7 37
9 36
11 35
13 34
15 33
17 32
19 31
21 30
23 29
25 28
27 27
29 26
31 25
33 24
35 23
37 22
39 21
41 20
43 19
45 18
47 17
49 16
42 22
44 21
46 20
48 19
50 18
52 17
54 16
56 15
58 14
60 13
62 12
64 11
66 10
68 9
70 8
72 7
74 6
76 5
78 4
80 3
82 2
0 85
2 84
81 4
83 3
85 2
0 88
2 87
4 86
6 85
8 84
10 83
12 82
14 81
16 80
18 79
20 78
22 77
24 76
26 75
28 74
30 73
32 72
34 71
36 70
38 69
31 75
33 74
35 73
37 72
39 71
41 70
43 69
45 68
47 67
49 66
51 65
53 64
55 63
57 62
59 61
61 60
63 59
65 58
67 57
69 56
71 55
73 54
75 53
68 59
70 58
72 57
74 56
76 55
78 54
80 53
82 52
84 51
86 50
88 49
90 48
92 47
94 46
96 45
98 44
100 43
102 42
104 41
106 40
108 39
110 38
112 37
105 43
107 42
109 41
111 40
113 39
115 38
117 37
119 36
121 35
123 34
125 33
127 32
129 31
131 30
133 

### Part 2

In [106]:
x = [0, 1, 2]
x.insert(2, 3)
x

[0, 1, 3, 2]

In [220]:
23 * 2 - 7

how_far_can_we_go = 

moves_remaining = moves - 

39